In [ ]:
import requests
from bs4 import BeautifulSoup
import scipy.io
import matplotlib.pyplot as plt
import matplotlib 
import pandas as pd
import numpy as np
import pickle
from time import sleep
import timeit
import json
import os, binascii
import hashlib
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
import pycountry_convert as pc


def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

        
def request(msg, slp=0.5):
    # A wrapper to make robust https requests.
    
    # Want to get a status-code of 200
    status_code = 500  
    i = 4
    while status_code != 200:
        # Don't ping the server too often
        sleep(slp)  
        try:
            r = requests.get(msg)
            status_code = r.status_code
            i -= 1
            #if status_code != 200:
                #print('Server Error! Response Code %i. Retrying...' % (r.status_code))
            if i == 0:
                print('Server Error! Response Code %i. Retrying...' % (r.status_code))
                return 0
        except:
            print('An exception has occurred, probably a momentary loss of connection. Waiting one seconds...')
            sleep(1)
    return r


# Data to be written
users = set()

csv = pd.read_csv('bgg_gamelist.csv')

w = 1

for i in range(0,17):

    r = request('https://boardgamegeek.com/xmlapi2/forumlist?id=%i&type=thing' % (csv['id'][i],))
    
    if r == 0:
        continue
    
    soup = BeautifulSoup(r.text, "xml")
    
    tmp = soup.find_all('forum')
    forums = []
    
    for forum in tmp:        
        r = request('https://boardgamegeek.com/xmlapi2/forum?id=%i' % (int(forum['id']),))
        
        if r == 0:
            continue
        
        soup = BeautifulSoup(r.text, "xml")
        
        threads = soup.find_all('thread')
        
        for thread in threads:            
            r = request('https://boardgamegeek.com/xmlapi2/thread?id=%i' % (int(thread['id']),))
            
            if r == 0:
                continue
            
            soup = BeautifulSoup(r.text, "xml")
            
            post = {}
            posts = soup.find_all('article')
                        
            for p in posts:
                r = request('https://boardgamegeek.com/xmlapi2/user?name=' + p['username'])

                if r == 0:
                    continue
                    
                soup = BeautifulSoup(r.text, "xml")
                
                tmp = users.copy()
                users.add(soup.user['id'])
                
                if tmp == users:
                    print('User ' + soup.user['name'] + ' already scraped')
                else:
                    print('User ' + soup.user['name'] + ' has just been scraped')
                    
                    passwd = ('_' + p['username'] + '_').encode("utf8")
                    hashed_passwd = hashlib.sha256(passwd).hexdigest()
                    
                    tmp_user = soup.find_all('user')
                    
                    user = {
                        'username'         : soup.user['name'],
                        'password'         : hashed_passwd,
                        'most_recent_post' : []
                    }
                    
                    # print(soup.user.firstname['value'])
                    if soup.user.firstname['value'] != '':
                        user['firstname'] = soup.user.firstname['value']
                        
                    # print(soup.user.lastname['value'])
                    if soup.user.lastname['value'] != '':
                        user['lastname'] = soup.user.lastname['value']
                    
                    # print(soup.user.yearregistered['value'])
                    if soup.user.yearregistered['value'] != '':
                        user['yearregistered'] = soup.user.yearregistered['value']
                    
                    # print(soup.user.stateorprovince['value'])
                    if soup.user.stateorprovince['value'] != '':
                        user['stateorprovince'] = soup.user.stateorprovince['value']
                    
                    # print(soup.user.avatarlink['value'])
                    if soup.user.avatarlink['value'] != '' and soup.user.avatarlink['value'] != 'N/A':
                        user['img'] = soup.user.avatarlink['value']
                    
                    # print(soup.user.country['value'])
                    if soup.user.country['value'] != '':
                        try:
                            user['continent'] = country_to_continent(soup.user.country['value'])
                        except:
                            print('Something goes wrong - Skipping continent regarding user ' + str(soup.user['name']))
                            
                    # print(user)
                                        
                    with open('user.json', 'a') as outfile:
                        if w == 0:
                            outfile.write(',')
                        else:
                            w -= 1
                        json.dump(user, outfile)

User BoredGamesLab has just been scraped
User whac3 has just been scraped
User Stephanie Baynes has just been scraped
User 1qwerty has just been scraped
User whac3 already scraped
User Gaucho80 has just been scraped
User BoredGamesLab already scraped
User whac3 already scraped
User JustinRRizzo has just been scraped
User Gaucho80 already scraped
User BoredGamesLab already scraped
User whac3 already scraped
User Gaucho80 already scraped
User 1qwerty already scraped
User BoredGamesLab already scraped
User Garzant has just been scraped
User Dormin87 has just been scraped
User Dkaratekid has just been scraped
User sgclouthier has just been scraped
User PapaGeek2 has just been scraped
User Fishticuffs has just been scraped
Something goes wrong - Skipping continent regarding user Fishticuffs
User markrollings has just been scraped
User DBear has just been scraped
User Gaucho80 already scraped
User northjack has just been scraped
User Gaucho80 already scraped
User northjack already scraped
Us

User SER BARRISTAN SELMY has just been scraped
User emoska1 already scraped
User morningview66 has just been scraped
User emoska1 already scraped
User Captain Maybe has just been scraped
User powerofzero has just been scraped
User TrippleB9 has just been scraped
User chriswray84 has just been scraped
User EMBison has just been scraped
User Vladvonbounce already scraped
User pandoraschoice has just been scraped
User Whangdoodle already scraped
User pandoraschoice already scraped
User markgravitygood already scraped
User pandoraschoice already scraped
User markgravitygood already scraped
User chrisoc13 has just been scraped
User Whaleyland already scraped
User chrisoc13 already scraped
User Ken at Sunrise has just been scraped
User elviernestocajugar has just been scraped
User bridgetvh has just been scraped
User LordBalder has just been scraped
User Borodin has just been scraped
User Wilseph already scraped
User koreanpenguin has just been scraped
User pricero1 has just been scraped
Use

User bholle has just been scraped
User Hayzuss has just been scraped
User Wired_Wolf has just been scraped
User DreadPirateRobert has just been scraped
User Gruumsh has just been scraped
User Hayzuss already scraped
User Karlsen has just been scraped
User Hayzuss already scraped
User Gruumsh already scraped
User Tryken has just been scraped
User ironman99 has just been scraped
User swheelock already scraped
User Elocution Safari has just been scraped
User nickster1970 has just been scraped
User house14 has just been scraped
User AceRandahl has just been scraped
User CrankyDay has just been scraped
User Amiguero already scraped
User swheelock already scraped
User Hockey Mask has just been scraped
User Amiguero already scraped
User conky has just been scraped
User Amiguero already scraped
User Amiguero already scraped
User muzfish4 has just been scraped
User xldh has just been scraped
User Neil Thomson already scraped
User house14 already scraped
User Amiguero already scraped
User Amigue

User lgoldberg already scraped
User khornschemeier already scraped
User Broadstorm already scraped
User TrippleB9 already scraped
User swheelock already scraped
User TVis already scraped
User Broadstorm already scraped
User erikieperikie already scraped
User TrippleB9 already scraped
User TVis already scraped
User TVis already scraped
User TrippleB9 already scraped
User TVis already scraped
User Gaucho80 already scraped
User TVis already scraped
User Gaucho80 already scraped
User monteslu has just been scraped
User Gaucho80 already scraped
User Grape Ape already scraped
User TrippleB9 already scraped
User kdba has just been scraped
User RockCartel has just been scraped
User RockCartel already scraped
User monteslu already scraped
User TVis already scraped
User lgoldberg already scraped
User ackmondual already scraped
User Broadstorm already scraped
User John_VW has just been scraped
User Booch77 already scraped
User cicerunner already scraped
User leemc13 already scraped
User John_VW a

User elhebert has just been scraped
User hirsty has just been scraped
User elhebert already scraped
User Dielotski has just been scraped
User giantmike has just been scraped
User Dielotski already scraped
User rolaids24 has just been scraped
User Grape Ape already scraped
User SuperGLS already scraped
User kissgg has just been scraped
User 1qwerty already scraped
User Dielotski already scraped
User jimmcmahon has just been scraped
User cicerunner already scraped
User Arctic Jack has just been scraped
Something goes wrong - Skipping continent regarding user Arctic Jack
User Dielotski already scraped
User Schoolmarm5 already scraped
User Dielotski already scraped
User VeilOfMaya has just been scraped
User Dielotski already scraped
User kolvorok has just been scraped
User northjack already scraped
User kolvorok already scraped
User cicerunner already scraped
User kolvorok already scraped
User Decar already scraped
User kolvorok already scraped
User Decar already scraped
User kolvorok alre

...

User privateer4hire has just been scraped
User David Goldfarb already scraped
User paulidale has just been scraped
User privateer4hire already scraped
User Omahavice already scraped
User casualgod already scraped
User Mamadallama already scraped
User porkchop_tw already scraped
User David Goldfarb already scraped
User roger1818 has just been scraped
User casualgod already scraped
User casualgod already scraped
User David Goldfarb already scraped
User Omahavice already scraped
User casualgod already scraped
User David Goldfarb already scraped
User casualgod already scraped
User Si Fei already scraped
User cerulean has just been scraped
User DaviddesJ already scraped
User DaviddesJ already scraped
User chadw09 already scraped
User Vetinari already scraped
User cerulean already scraped
User cerulean already scraped
User banyan already scraped
User VinceHerman has just been scraped
User cerulean already scraped
User familygaming has just been scraped
User dabowsa has just been scraped
User

User BaconSnake has just been scraped
User royal tenenbaum has just been scraped
User audreypots has just been scraped
User BaconSnake already scraped
User Squidd already scraped
User Seen already scraped
User Omahavice already scraped
User Stunna already scraped
User shrimple has just been scraped
User Beaveman already scraped
User pratchettfan777 already scraped
User Ripshawd already scraped
User xpiredsodapop already scraped
User Hockey Mask already scraped
User xpiredsodapop already scraped
User Hockey Mask already scraped
User xpiredsodapop already scraped
User Octavian already scraped
User donaldx already scraped
User JustinRRizzo already scraped
User 1qwerty already scraped
User JustinRRizzo already scraped
User nimzo28 already scraped
User 1qwerty already scraped
User TheJRMY already scraped
User Hockey Mask already scraped
User ackmondual already scraped
User johoohno has just been scraped
User TheJRMY already scraped
User Harlandas has just been scraped
User tylorlilley has j

User caresrg already scraped
User sumpfork already scraped
User zombie13 has just been scraped
User HarryCanyon has just been scraped
User sumpfork already scraped
User sumpfork already scraped
User sumpfork already scraped
User sumpfork already scraped
User timdogg1934 has just been scraped
User nickv2002 already scraped
User The Broox already scraped
User johnostout already scraped
User pacatak already scraped
User nickv2002 already scraped
User sumpfork already scraped
User rickert already scraped
User nickv2002 already scraped
User strobech has just been scraped
User sumpfork already scraped
User pacatak already scraped
User belltower has just been scraped
User nickv2002 already scraped
User smwoz17 has just been scraped
User nickv2002 already scraped
User zarfin has just been scraped
User rogue148 already scraped
User zarfin already scraped
User sumpfork already scraped
User rogue148 already scraped
User sumpfork already scraped
User rogue148 already scraped
User sumpfork already 

User LastFootnote already scraped
User darthnice already scraped
User natty_n8 already scraped
User sr71woodworks has just been scraped
User thegreybetween already scraped
User stoneleaf already scraped
User sr71woodworks already scraped
User thewagon already scraped
User tabicat already scraped
User sr71woodworks already scraped
User ovka has just been scraped
User TrippleB9 already scraped
User MontyBell has just been scraped
User severly has just been scraped
User irianamistifi has just been scraped
User natty_n8 already scraped
User jammasterjake already scraped
User HappyProle already scraped
User Spielguy already scraped
User Grudunza already scraped
User jammasterjake already scraped
User Spielguy already scraped
User theory already scraped
User HappyProle already scraped
User hobbes_inc already scraped
User Saan has just been scraped
User wcbarksdale already scraped
User fiddlerjones already scraped
User destry already scraped
User HappyProle already scraped
User HappyProle alr

User Sappington already scraped
User MasterDinadan already scraped
User tokentoken has just been scraped
User Mxrk has just been scraped
User Picon already scraped
User Arkhamssaber has just been scraped
User salva already scraped
User steckman already scraped
User JackRudd already scraped
User rhubarbjin has just been scraped
User bmacfarlane has just been scraped
User jaeppel has just been scraped
User jaeppel already scraped
User carnalito already scraped
User Robledo already scraped
User thutson has just been scraped
User johnweldy already scraped
User MunchWolf already scraped
User MunchWolf already scraped
User MD1616 already scraped
User johnweldy already scraped
User LastFootnote already scraped
User thutson already scraped
User thutson already scraped
User madantichris already scraped
User johnweldy already scraped
User madantichris already scraped
User teg2 has just been scraped
User johnweldy already scraped
